##Script Setup:

    export HOME=/home/ub2
    export SERVER=SERVER2

##Prerequisites:

    sudo apt-get install -y \
        libdrizzle-dev \
        postgresql-server-dev-9.3 \
        libpq-dev \
        libpqxx-dev \
        lua-sql-postgres-dev \
        pgagent \
        postfix-pgsql \
        postgis \
        postgresql-9.3-ip4r \
        postgresql-9.3-plsh \
        postgresql-9.3-postgis-scripts \
        postgresql-client-common \
        postgresql-comparator \
        postgresql-contrib-9.3 \
        postgresql-prioritize \
        postgresql-plpython-9.3 \
        postgresql-plpython3-9.3 || exit 1

##LibDrizzle-1.0

    wget http://openresty.org/download/drizzle7-2011.07.21.tar.gz || exit 1

    tar xzvf drizzle7-2011.07.21.tar.gz || exit 1
    cd drizzle7-2011.07.21/

    sudo apt-get install -y \
        intltool \
        libzthread-dev \
        uuid-dev \
        libreadline-dev \
        libprotobuf-dev \
        protobuf-compiler \
        libprotobuf-dev \
        libboost1.55-all-dev \
        libboost1.55-tools-dev \
        libcurl4-gnutls-dev \
        libgcrypt11-dev || exit 1

    ./configure --without-server && sudo make libdrizzle-1.0 && sudo make install-libdrizzle-1.0 || exit 1


## OpenResty

####Download latest version:

    wget http://openresty.org/download/ngx_openresty-1.7.7.1.tar.gz || exit 1
    tar -zxvf ngx_openresty-1.7.7.1.tar.gz || exit 1
    cp -R mods ngx_openresty-1.7.7.1/ || exit 1
    cd ngx_openresty-1.7.7.1

####Configure

    sudo ./configure \
        --with-pcre-jit \
        --with-ipv6 \
        --with-debug \
        --with-luajit \
        --with-http_drizzle_module \
        --with-http_iconv_module \
        --with-libdrizzle=/usr/local \
        --with-http_postgres_module \
        --with-select_module \
        --add-module=mods/nginx-http-rdns-master \
        --add-module=mods/ngx_aws_auth-master \
        --add-module=mods/ngx_http_lower_upper_case-master \
        --add-module=mods/ngx_http_internal_redirect-master \
        --add-module=mods/ngx_upstream_jdomain-master \
        --with-http_sub_module \
        --with-http_addition_module \
        --with-http_spdy_module \
        --with-http_ssl_module \
        --error-log-path=logs/ngx_error.log || exit 1

####Install:

    sudo make && sudo make install || exit 1
    sudo unlink /usr/local/bin/nginx
    sudo ln -s /usr/local/openresty/nginx/sbin/nginx /usr/local/bin/ || exit 1
    cd ..

####Add Lua-resty-logger-socket:

    git clone --recursive https://github.com/cloudflare/lua-resty-logger-socket.git || exit 1
    sudo mkdir /usr/local/openresty/lualib/resty_logger || exit 1
    sudo mv lua-resty-logger-socket/lib/resty/logger /usr/local/openresty/lualib/resty/ || exit 1

### NGX Settings:

    mv -f $HOME/$SERVER/nginx/setup/nginx/conf/nginx.conf $HOME/$SERVER/nginx/
    mv $HOME/$SERVER/nginx/setup/nginx/sites-available $HOME/$SERVER/nginx/
    rm -R $HOME/$SERVER/nginx/setup/nginx
    cp -R /usr/local/openresty/nginx  $HOME/$SERVER/nginx/setup
    rm -fR $HOME/$SERVER/nginx/setup/nginx/sites-available

    mv -f $HOME/$SERVER/nginx/nginx.conf $HOME/$SERVER/nginx/setup/nginx/conf/
    mv -f $HOME/$SERVER/nginx/sites-available $HOME/$SERVER/nginx/setup/nginx/

###Clean-up:

    sudo rm -R drizzle7-2011.07.21.tar.gz ngx_openresty-1.7.7.1.tar.gz
    sudo rm -R drizzle7-2011.07.21 lua-resty-logger-socket ngx_openresty-1.7.7.1